# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "mtecbo"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-25 10:26:47,333 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("CODIGO", T.StringType(), True),
    T.StructField("TITULO", T.StringType(), True)
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("mte/cbo")
local = f"s3a://landing/mte/cbo/{ano_mes}/CBO2002 - Familia.csv"

In [5]:
config_adicional = {
    'header': True,
    'sep': ';',
    'encoding': 'iso-8859-1'
}

# Leitura do arquivo

In [6]:
df_compliance_cbo_familia = extrator_bronze.criar_view_temporaria_arquivo(local,'compliance_cbo_familia', schema,**config_adicional)

2024-10-25 10:26:49,276 - INFO - Lista de yaml carregada com sucesso
2024-10-25 10:26:51,158 - INFO - Criando view temporaria.


In [7]:
sql_query = """
    SELECT *,
           substring(codigo, 1, 3) AS cd_subgrupo
    FROM compliance_cbo_familia
"""

In [8]:
df_compliance_cbo_sub_grupo = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-25 10:26:51,180 - INFO - Aguarde... Executando Query (Delta)
2024-10-25 10:26:51,180 - INFO - 
    SELECT *,
           substring(codigo, 1, 3) AS cd_subgrupo
    FROM compliance_cbo_familia

2024-10-25 10:26:51,353 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [9]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/mte/cbofamilia'

In [10]:
extrator_bronze.salvar_delta(df_compliance_cbo_sub_grupo, 'overwrite')

2024-10-25 10:26:51,366 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-25 10:26:57,946 - INFO - Dados persistidos com sucesso
2024-10-25 10:26:57,947 - INFO - s3a://bronze/mte/cbofamilia
2024-10-25 10:26:57,948 - INFO - Aguarde... Realizando optimize
2024-10-25 10:27:02,579 - INFO - Optimize executado com sucesso.
2024-10-25 10:27:02,581 - INFO - Aguarde... Realizando vacuum
2024-10-25 10:27:18,437 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [11]:
extrator_bronze.parar_sessao()

2024-10-25 10:27:18,660 - INFO - Sessão Spark finalizada.
